In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, SQLContext

from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import pandas_udf
from pyspark.sql.window import Window

import pandas as pd
import numpy as np

from datetime import date, datetime, timedelta, timezone
import os
import json
import requests
import time
import yaml
from itertools import chain

In [2]:
appName = "PySpark TFT puuids"
master = "local[*]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.executor.memory", "40g") \
    .set("spark.driver.memory", "40g") \
    .set("spark.executor.memoryOverhead", "8g") \
    .set("spark.local.dir", "/home/mai/spark-temp") \
    .set("spark.sql.session.timeZone", "UTC") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.minExecutors", "2") \
    .set("spark.dynamicAllocation.maxExecutors", "50") \
    .set("spark.speculation", "true") 
   
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

25/02/07 10:44:00 WARN Utils: Your hostname, LAPTOP-4O0SI9BK resolves to a loopback address: 127.0.1.1; using 172.26.81.220 instead (on interface eth0)
25/02/07 10:44:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/07 10:44:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/07 10:44:02 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Set up

In [3]:
with open('./api_key.yaml', 'r') as file:
    API_KEY = yaml.safe_load(file)['api_key']

In [4]:
API_KEY

'RGAPI-c221510b-d09b-4a40-bccd-7d74c5bb5b35'

## Get match history (most recent 100 matches)
We take the most recent 100 matches and then later we will filter out only the ranked TFT matches

In [5]:
REGION = 'kr'
REGION_TO_EXECUTE = 'ASIA'
BASE_URL = f'https://{REGION_TO_EXECUTE}.api.riotgames.com/tft/match/v1/matches/by-puuid'
TIER = 'GOLD'
DIVISION = 'I'
NUMBER_OF_MATCHES = 100
# patch 13.4
PATCH_START_TIME = '1737590400'
PATCH_END_TIME = '1738713599'

In [6]:
# get the puuid
# df = pd.read_parquet(f'./data/tft_puuids/puuids_{REGION}_{TIER}_{DIVISION}.parquet')
df = pd.read_parquet(f'./data/tft_players/players_{REGION}_{TIER}_{DIVISION}.parquet')

In [7]:
df.head()

,puuid,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,uJr5xHJWi1YbxEjmAbX7F_8wPAUTKyS9MQlU9JENF7DhIU...,5f38105e-3c70-4f73-a80d-0d53e263f9bf,RANKED_TFT,GOLD,I,OGh2Z6KNBeg1vwDT0775MA-GUhockO1QlWGjEQjYs3RlYvc0,24,94,106,False,False,False,False
1,UKNPf16zVG0-hn5m4Q7bRT3dgPBHtjddxTdoP-jy_Bsm7x...,c003cdd3-1b68-4190-8e7c-1470a9ce1819,RANKED_TFT,GOLD,I,ww9_0qou-pPMD5asdz3XlXUrxX1VJQ87CH5N_nrZVQzYip5h,32,27,24,False,False,False,False
2,wogzb-1g1KxdC9r1pYaBwy0MpmfnX53NfQPMWdkyiS7atM...,bf5f46cc-6735-436b-820b-ceaa55e4a415,RANKED_TFT,GOLD,I,zxUZvHcJ_lI4sZ0Tj6aA4XPHsdQ9NXJYixGSla7stF02PeYA,12,33,29,False,False,False,False
3,aumJ0m9ngyLklNA2Z6DVk2MArXZK_VJX8J477BxbbfuBbu...,eaf9004c-1551-487a-aab4-7740bda441a9,RANKED_TFT,GOLD,I,1ICDes8AlSNGsxysXu-E5mbms8nout9AzAboZPPfKFs4Q7T7,0,37,35,False,False,False,False
4,pofPVwkyQrTQWOzsGTpxiJMq64AM7OuzAhjL5SOglqivyB...,50677d4b-fad4-489a-aef3-86b8312f71d4,RANKED_TFT,GOLD,I,z77ioRDuUwB8WmNmJ3QfhYE_-ssK1GO8eAtQcL-gBME6wk0o,24,79,77,False,False,False,True


In [8]:
puuids = df['puuid']

In [9]:
len(puuids)

1025

In [10]:
puuids[0]

'uJr5xHJWi1YbxEjmAbX7F_8wPAUTKyS9MQlU9JENF7DhIUIH9cskcJiXaBuWd_n0QkfqEvD2DKplNw'

In [13]:
def get_match_history(puuids_, n_matches=100, start=0, end_time=PATCH_END_TIME, start_time=PATCH_START_TIME):
    match_list = []
    i = 0
    while i < len(puuids_):
        # construct url
        url = f'{BASE_URL}/{puuids_[i]}/ids?start={start}&endTime={end_time}&startTime={start_time}&count={n_matches}'
        # headers
        headers = {
            'X-Riot-Token': API_KEY,
        }
        # send request
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            # parse json response
            print(i)
            data = response.json()
            match_list = list(dict.fromkeys(match_list + data))
            # match_history_data.append(data)
            time.sleep(1.2)
            i += 1
        elif response.status_code == 429:
            print("Rate limit exceeded. Waiting before retrying...")
            # Extract "Retry-After" header if available
            retry_after = int(response.headers.get("Retry-After", 10))
            time.sleep(retry_after)  # Wait for the specified time
        else:
            print(f'Error: {response.status_code} - {response.text}')
            # i += 1
            break
    return match_list

In [14]:
match_hist = get_match_history(puuids)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
len(match_hist)

21256

In [16]:
match_hist_df = pd.DataFrame(match_hist, columns=['match_id'])
match_hist_df = spark.createDataFrame(match_hist_df)
match_hist_df.show(10)

+-------------+
|     match_id|
+-------------+
|KR_7507688345|
|KR_7507470527|
|KR_7507423268|
|KR_7507343876|
|KR_7507292029|
|KR_7507061593|
|KR_7507009076|
|KR_7506632413|
|KR_7506504490|
|KR_7506231057|
+-------------+
only showing top 10 rows



In [17]:
match_hist_df.write.mode('overwrite').parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}.parquet')